<a href="https://colab.research.google.com/github/vanecornejo/EDP-II/blob/main/M%C3%A9todo%20de%20Gauss%20Seidel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Método de Gauss-Seidel aplicado a la Ecuación de Laplace**
 El método de Gauss-Seidel es una técnica iterativa para resolver sistemas lineales de la forma:  
$$ A x = b $$
La idea principal es aproximar la solución empezando desde una estimación inicial y actualizando los valores en cada paso, utilizando los resultados más recientes disponibles.  
La fórmula de actualización para el elemento $x_i$ es:

$$
x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} \right)
$$


## Implementando el problema

Implementando el segundo ejercicio del problemario,

Queremos aproximar la solución de la ecuación de Laplace:

$$ \nabla^2 u(x,y) = 0 $$

con las siguientes condiciones de frontera en la región cuadrada $ 0\leq x,y \leq 2 $:  

- $ u(0,y) = 0 $  
- $ u(2,y) = y(2-y) $, para \(0 < y < 2\)  
- $ u(x,0) = 0 $  
- $ u(x,2) =
\begin{cases}
x & \text{si } 0 < x < 1 \\
2-x & \text{si } 1 \leq x < 2
\end{cases}
$

El tamaño de la malla es:

$$ h = 0.5 $$

Esto divide la región en una grilla de 5 × 5 puntos:  

$$
x = 0, 0.5, 1, 1.5, 2, \quad y = 0, 0.5, 1, 1.5, 2
$$


## Ecuación en los puntos interiores

Para cada punto interior $(i,j)$, la ecuación de Laplace discretizada se convierte en:

$$
u_{i,j} = \frac{1}{4} \left( u_{i-1,j} + u_{i+1,j} + u_{i,j-1} + u_{i,j+1} \right)
$$

Es decir, el valor en un nodo interior es el promedio de sus 4 vecinos.


## Solución

1. Definimos la malla con $h=0.5$  
2. Aplicamos las condiciones de frontera en los bordes
3. Inicializamos los valores interiores con ceros
4. Usamos el método de Gauss-Seidel para actualizar los valores de los nodos interiores hasta que la solución converja con una tolerancia dada
5. Mostramos los resultados en la tabla  

In [1]:
# Importamos librerías
import numpy as np
import pandas as pd

Ponemos los datos de nuestro ejercicio

In [2]:
# Parámetros de la malla
h = 0.5
n = int(2/h) + 1   # número de puntos en cada eje
x = np.linspace(0, 2, n)
y = np.linspace(0, 2, n)

# Matriz solución inicial
U = np.zeros((n, n))

# Condiciones de frontera:
U[:,0] = 0   # u(0,y)=0
U[:,-1] = y*(2-y)   # u(2,y)=y(2-y)
U[0,:] = 0   # u(x,0)=0

# Para u(x,2) = {x si 0<x<1 ; 2-x si 1<=x<2}
for j in range(n):
    if x[j] < 1:
        U[-1,j] = x[j]
    else:
        U[-1,j] = 2 - x[j]

Aplicando el Método de Gauss-Seidel

In [3]:
tol = 1e-4
max_iter = 5000

for it in range(max_iter):
    error = 0.0
    for i in range(1, n-1):
        for j in range(1, n-1):
            old = U[i,j]
            U[i,j] = 0.25 * (U[i-1,j] + U[i+1,j] + U[i,j-1] + U[i,j+1])
            error = max(error, abs(U[i,j]-old))
    if error < tol:
        print(f"Convergencia alcanzada en {it+1} iteraciones")
        break
else:
    print("Número máximo de iteraciones alcanzado")

df = pd.DataFrame(U, index=[f"y={yi:.1f}" for yi in y],
                     columns=[f"x={xi:.1f}" for xi in x])

print(df)

Convergencia alcanzada en 14 iteraciones
       x=0.0     x=0.5     x=1.0     x=1.5  x=2.0
y=0.0    0.0  0.000000  0.000000  0.000000   0.00
y=0.5    0.0  0.109316  0.227619  0.395060   0.75
y=1.0    0.0  0.209762  0.406191  0.602649   1.00
y=1.5    0.0  0.323631  0.584792  0.609360   0.75
y=2.0    0.0  0.500000  1.000000  0.500000   0.00
